In [3]:
#import tensorflow as tf
import datetime
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import TensorBoard
import os, shutil
from PIL import Image, ImageDraw, ImageFont, ImageColor
import random
import pandas as pd

In [4]:
%reload_ext tensorboard

In [5]:
train_dataset_dir ='\\Users\\danie\\Penn_State_REU_Jupyter\\Hexagon ML Project\\Training_set_6_10' 

df=pd.read_csv(os.path.join(train_dataset_dir,"training_database.csv"))
df = df.sample(frac=1).reset_index(drop=True)

In [6]:
#df.shape
print(df)

filename   x_coord   y_coord
0     bl8_offset0_as1.png  0.500000  0.433013
1     bl6_offset6_as1.png  0.468750  0.487139
2     bl6_offset7_as2.png  0.437500  0.487139
3     bl7_offset5_as2.png  0.609375  0.568329
4    bl8_offset13_as1.png  0.593750  0.433013
..                    ...       ...       ...
115   bl8_offset7_as1.png  0.406250  0.649519
116   bl9_offset1_as2.png  0.531250  0.487139
117  bl6_offset13_as2.png  0.625000  0.487139
118   bl7_offset2_as1.png  0.484375  0.568329
119   bl8_offset5_as1.png  0.468750  0.649519

[120 rows x 3 columns]


In [7]:
def build_model():
    model = models.Sequential()                                  
    model.add(layers.Conv2D(16, (3,3) ,activation='relu', input_shape=(32,32,3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(16,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(2))
    return model

In [8]:
#Creates generators for training and validation images
SEED=5
validation_split_frac=0.2
batch_size=2
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=validation_split_frac) 
train_generator= train_datagen.flow_from_dataframe(
        df,
        directory=train_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord'],
        target_size=(32,32),
        shuffle=True,
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size,
        subset='training')

validation_generator= train_datagen.flow_from_dataframe(
        df,
        directory=train_dataset_dir,
        x_col="filename",
        y_col=['x_coord','y_coord'],
        target_size=(32,32),
        shuffle=True,
        seed= SEED,
        class_mode="raw",
        batch_size=batch_size,
        subset='validation')

Found 96 validated image filenames.
Found 24 validated image filenames.


In [14]:
model=build_model()
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])
#model.summary()
num_epochs=200
training_steps=int(df.shape[0]*(1-validation_split_frac)/batch_size)
validation_steps=int(df.shape[0]*validation_split_frac/batch_size)
assert (training_steps+validation_steps)*batch_size == df.shape[0]
log_dir = os.path.join("logs\\fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(
    train_generator,
    steps_per_epoch=training_steps,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=num_epochs,
    verbose=1,
    callbacks=[tensorboard_callback]
    )


=================] - 0s 5ms/step - loss: 0.0032 - mae: 0.0437 - val_loss: 0.0060 - val_mae: 0.0625
Epoch 51/200
48/48 [==============================] - 0s 5ms/step - loss: 0.0031 - mae: 0.0434 - val_loss: 0.0050 - val_mae: 0.0512
Epoch 52/200
48/48 [==============================] - 0s 5ms/step - loss: 0.0030 - mae: 0.0434 - val_loss: 0.0057 - val_mae: 0.0597
Epoch 53/200
48/48 [==============================] - 0s 5ms/step - loss: 0.0030 - mae: 0.0413 - val_loss: 0.0051 - val_mae: 0.0557
Epoch 54/200
48/48 [==============================] - 0s 5ms/step - loss: 0.0032 - mae: 0.0436 - val_loss: 0.0049 - val_mae: 0.0509
Epoch 55/200
48/48 [==============================] - 0s 6ms/step - loss: 0.0030 - mae: 0.0421 - val_loss: 0.0058 - val_mae: 0.0566
Epoch 56/200
48/48 [==============================] - 0s 5ms/step - loss: 0.0029 - mae: 0.0417 - val_loss: 0.0048 - val_mae: 0.0502
Epoch 57/200
48/48 [==============================] - 0s 5ms/step - loss: 0.0029 - mae: 0.0410 - val_loss: 0.

In [9]:
model=build_model()
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])
#model.summary()

In [12]:
from tensorboard import notebook
notebook.list()
%tensorboard --logdir Training_set_6_10/logs/fit

Known TensorBoard instances:
  - port 6006: logdir logs/fit (started 1 day, 0:30:22 ago; pid 10216)
  - port 6006: logdir logs/fit (started 4:52:26 ago; pid 10656)
  - port 6006: logdir Training_set_6_10/logs/fit (started 0:07:39 ago; pid 13120)
  - port 6006: logdir Training_set_6_10/logs/fit (started 0:02:04 ago; pid 15348)
  - port 6006: logdir logs (started 0:30:34 ago; pid 16588)


Reusing TensorBoard on port 6006 (pid 15348), started 0:02:04 ago. (Use '!kill 15348' to kill it.)

In [ ]:

#Loads labels and training data into two arrays, such that their indicies match
# train_labels=[]
# with open("Training_set_6_10\\training_labels.json",'r') as dict_contents:
#     train_labels_dict=json.load(dict_contents)
# train_labels= [value for key,value in sorted(train_labels_dict.items())]

# train_images1 = []
# path = original_dataset_dir
# valid_images = [".jpg",".gif",".png",".tga"]
# for f in os.listdir(path):
#     ext = os.path.splitext(f)[1]
#     if ext.lower() not in valid_images:
#         continue
#     #print(f[10:-4])
#     train_images1.append(Image.open(os.path.join(path,f)))
#     train_targets1.append(train_targets_dict[str(f[10:-4])])


In [ ]:
# k = 4
# num_val_samples = len(train_images) // k
# num_epochs = 20
# all_histories = []
# train_datagen = ImageDataGenerator(rescale=1./255)             
# val_datagen = ImageDataGenerator(rescale=1./255) 
# for i in range(k):
#     print('processing fold #', i)
#     val_images = train_images[i * num_val_samples: (i + 1) * num_val_samples]    
#     val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

#     partial_train_images = train_images[:i * num_val_samples] + train_images[(i + 1) * num_val_samples:]
#     partial_train_targets = train_targets[:i * num_val_samples] + train_targets[(i + 1) * num_val_samples:]
    
#     train_generator= train_datagen.flow(
#         partial_train_images,
#         y=partial_train_targets,
#         shuffle=True,
#         batch_size=1)
#     val_generator= val_datagen.flow(
#         val_images,
#         y=val_targets,
#         shuffle=True,
#         batch_size=1)

#     model = build_model()                                                    
#     history= model.fit_generator(
#         train_generator,
#         steps_per_epoch=len(partial_train_images),
#         epochs=num_epochs,
#         validation_data=val_generator,
#         validation_steps=num_val_samples)
#     #mae_history = history.history['val_mean_absolute_error']
#     all_histories.append(history)

In [ ]:
# #index_list=random.sample(range(len(train_images)),len(train_images))
# index_list=[12, 77, 11, 32, 35, 28, 58, 70, 0, 46, 64, 74, 13, 1, 19, 2, 57, 39, 56, 30, 71, 49, 38, 7, 47, 75, 43, 25, 5, 44, 18, 50, 51, 17, 21, 27, 48, 29, 6, 10, 8, 34, 76, 9, 69, 60, 62, 15, 65, 53, 59, 16, 24, 4, 55, 22, 63, 78, 73, 67, 20, 42, 68, 52, 26, 79, 36, 33, 45, 41, 37, 72, 14, 40, 66, 3, 54, 31, 61, 23]
# train_images=[]
# train_targets=[]
# for i in index_list:
#     train_images.append(train_images1[i])
#     train_targets.append(train_targets1[i])


In [ ]:
# fig = plt.figure(figsize = (10, 10))
# ax = plt.subplot(1,1,1)
# ax.imshow(train_images1[3])
# ax.axis('off')